In [1]:
# from transformers import AutoTokenizer, AutoModel
# import torch
# import numpy as np

# device = "cuda" if torch.cuda.is_available() else "cpu"

# model_name = "NVIDIA/nv-embed-v2"

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
# model.to(device)
# model.eval()


Hàm chuẩn hóa lại các node

In [1]:
def format_node_text(node_name: str, node_type: str) -> str:
    raw = node_name.replace(f"{node_type}_", "").replace("_", " ")
    if node_type == "skill":
        return f"skill: {raw}"
    elif node_type == "domain":
        return f"domain: {raw}"
    elif node_type == "position":
        return f"job position: {raw}"
    elif node_type == "job_title":
        return f"job title: {raw}"
    else:
        return raw


In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')

e:\Spring Semester 2025\DWH&DM\DM\recomendation\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


emdedding các loại node skill, domain, position, job_title

In [5]:
import mysql.connector
import numpy as np
from tqdm import tqdm

# Kết nối tới MySQL
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='231123',
    database='cv'
)
cur = conn.cursor()

# Các loại node cần embedding
target_types = ("skill", "domain", "position", "job_title")
placeholders = ', '.join(['%s'] * len(target_types))
query = f"SELECT id, node_name, node_type FROM nodes WHERE node_type IN ({placeholders}) order by id LIMIT 15000,10000"
cur.execute(query, target_types)
rows = cur.fetchall()
# print(rows)
node_embeddings = {}

for node_id, node_name, node_type in tqdm(rows, desc="Embedding nodes"):
    try:
        text = format_node_text(node_name, node_type)
        emb = model.encode(text, normalize_embeddings=True)
        blob = emb.astype(np.float32).tobytes()

        # Cập nhật embedding vào bảng nodes
        cur.execute("""
            UPDATE nodes
            SET embedding = %s
            WHERE id = %s
        """, (blob, node_id))

    except Exception as e:
        print(f"❌ Lỗi tại node {node_name}: {e}")

conn.commit()
cur.close()
conn.close()



Embedding nodes: 100%|██████████| 36/36 [00:01<00:00, 34.00it/s]


In [ ]:
# import mysql.connector
# import numpy as np

# # Kết nối tới MySQL
# conn = mysql.connector.connect(
#     host='localhost',
#     user='root',
#     password='231123',
#     database='cv'
# )
# cur = conn.cursor()

# # Đọc embedding của 1 node bất kỳ (ví dụ node có id = 2022)
# node_id = 3
# cur.execute("SELECT embedding FROM nodes WHERE id = %s", (node_id,))
# result = cur.fetchone()

# if result and result[0] is not None:
#     blob = result[0]
#     emb = np.frombuffer(blob, dtype=np.float32)
#     print(f"✅ Vector embedding có kích thước: {emb.shape}")
# else:
#     print("⚠️ Không tìm thấy embedding cho node với ID:", node_id)

# cur.close()
# conn.close()


✅ Vector embedding có kích thước: (384,)


embedding các quan hệ

In [6]:
import mysql.connector
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# Hàm chuẩn hóa tên quan hệ
def format_relation_text(relation_name: str) -> str:
    return "relation: " + relation_name.replace("_", " ").lower()


# Kết nối MySQL
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='231123',
    database='cv'
)
cur = conn.cursor()

# Đọc tất cả relations
cur.execute("SELECT id, relation_name FROM relations ORDER BY id")
rows = cur.fetchall()

for rel_id, rel_name in tqdm(rows, desc="Embedding relations"):
    try:
        text = format_relation_text(rel_name)
        emb = model.encode(text, normalize_embeddings=True)
        blob = emb.astype(np.float32).tobytes()

        # Cập nhật embedding vào bảng relations
        cur.execute("""
            UPDATE relations
            SET embedding = %s
            WHERE id = %s
        """, (blob, rel_id))

    except Exception as e:
        print(f"❌ Lỗi tại relation {rel_name}: {e}")

conn.commit()
cur.close()
conn.close()


Embedding relations: 100%|██████████| 6/6 [00:00<00:00, 43.48it/s]


In [5]:
import mysql.connector
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# Hàm chuẩn hóa tên quan hệ
def format_relation_text(relation_name: str) -> str:
    return "relation: " + relation_name.replace("_", " ").lower()

# Khởi tạo model LLM
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')

# Kết nối MySQL
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='231123',
    database='cv'
)
cur = conn.cursor()
cur.execute("SELECT embedding FROM relations WHERE id = %s", (1,))
result = cur.fetchone()
if result and result[0]:
    vec = np.frombuffer(result[0], dtype=np.float32)
    print("✅ Vector quan hệ:", vec.shape)
else:
    print("⚠️ Không có embedding.")


✅ Vector quan hệ: (384,)


embedding cho các node candidate 

In [7]:
import mysql.connector
from tqdm import tqdm

# Kết nối tới MySQL
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='231123',
    database='cv'
)
cur = conn.cursor(dictionary=True)

# 1. Lấy danh sách candidate_id từ bảng nodes
cur.execute("SELECT node_name FROM nodes WHERE node_type = 'candidate'")
candidate_ids = [row['node_name'].replace('candidate_', '', 1) for row in cur.fetchall()]

# 2. Với mỗi candidate_id, lấy thông tin từ bảng candidate và experience
for cid in tqdm(candidate_ids, desc="Processing candidates"):
    # Lấy thông tin cơ bản từ bảng candidate
    cur.execute("SELECT * FROM candidate WHERE candidate_id = %s", (cid,))
    candidate_info = cur.fetchone()
    
    if not candidate_info:
        continue  # Nếu không có thông tin, bỏ qua

    # Lấy các kinh nghiệm từ bảng experience
    cur.execute("SELECT * FROM experience WHERE candidate_id = %s", (cid,))
    experiences = cur.fetchall()

    # 3. Generate candidate description in English
    desc = ""
    if candidate_info['position_title'] != None and len(candidate_info['position_title']) > 0:
        desc += f"The candidate is applying for the position of {candidate_info['position_title']},"
    if candidate_info['domain_area'] != None and len(candidate_info['domain_area']) > 0:
        desc += f" with expertise in the field of {candidate_info['domain_area']}."
    if candidate_info['skill'] != None and len(candidate_info['skill']) > 0:
        desc += f" General skills include: {candidate_info['skill']}."
    if experiences:
        desc += " Work experience: "
        for exp in experiences:
            
            if exp['title'] != None and len(exp['title']) > 0:
                desc += f"{exp['title']}"
            if exp['skill'] != None and len(exp['skill']) > 0:
                desc +=  f" using the skill {exp['skill']}"
            if exp['years_experience'] != None:
                desc += f" for {exp['years_experience']} years. "

    # print(cid, desc)
    # 4. Tính embedding
    embedding = model.encode(desc, normalize_embeddings=True)

    blob = embedding.astype(np.float32).tobytes()
    tmp = "candidate_"+cid
    # 5. Cập nhật embedding vào bảng nodes
    update_sql = "UPDATE nodes SET embedding = %s WHERE node_name = %s"
    cur.execute(update_sql, (blob, tmp))
    conn.commit()  # cần commit để lưu thay đổi

# Đóng kết nối
cur.close()
conn.close()


Processing candidates: 100%|██████████| 987/987 [01:55<00:00,  8.52it/s]


In [5]:
import mysql.connector
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Load model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Tạo mô tả ứng viên mới trong lĩnh vực y tế
desc = """
We are seeking a Medical Specialist with expertise in healthcare. The ideal candidate should have at least 5 years of experience as an Emergency Doctor with strong triage skills, and 3 years of experience as a Surgeon specializing in laparoscopic surgery. General skills required include emergency care, surgery, and patient management, along with the ability to perform effectively in high-pressure situations.
"""

# Tính embedding cho ứng viên mới
query_vec = model.encode(desc, normalize_embeddings=True)

# Kết nối DB
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='231123',
    database='cv'
)
cur = conn.cursor()

# Truy vấn embedding từ DB
cur.execute("SELECT node_name, embedding FROM nodes WHERE node_type = 'candidate' AND embedding IS NOT NULL")
rows = cur.fetchall()

# Danh sách tên và embeddings
names = []
vectors = []

for name, blob in rows:
    emb = np.frombuffer(blob, dtype=np.float32)
    names.append(name)
    vectors.append(emb)

vectors = np.vstack(vectors)  # convert to matrix

# Tính cosine similarity
similarities = cosine_similarity([query_vec], vectors)[0]

# Lấy 5 người giống nhất
top_indices = similarities.argsort()[-5:][::-1]

print("Top 5 most similar candidates:")
for i in top_indices:
    print(f"{names[i]} — similarity: {similarities[i]:.4f}")

# Đóng kết nối
cur.close()
conn.close()


Top 5 most similar candidates:
candidate_maarten-koole-a1640210 — similarity: 0.2959
candidate_xun-jia-8602655 — similarity: 0.2954
candidate_megane-guegan-a584a3193 — similarity: 0.2849
candidate_robertosterstatistics — similarity: 0.2816
candidate_john-pandolfino-67534919 — similarity: 0.2746


embedding cho job

In [8]:
import mysql.connector
from tqdm import tqdm
import numpy as np

# Connect to MySQL
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='231123',
    database='cv'
)
cur = conn.cursor(dictionary=True)

# Fetch data from all tables
cur.execute("SELECT * FROM nodes")
nodes = cur.fetchall()

cur.execute("SELECT * FROM relations")
relations = cur.fetchall()

cur.execute("SELECT * FROM edges")
edges = cur.fetchall()



# Create quick lookup dictionaries
node_dict = {node['id']: node for node in nodes}
relation_dict = {r['id']: r['relation_name'] for r in relations}

# Get specific relation IDs
HAS_TITLE_ID = next((r['id'] for r in relations if r['relation_name'] == 'HAS_TITLE'), None)
REQUIRES_SKILL_ID = next((r['id'] for r in relations if r['relation_name'] == 'REQUIRES_SKILL'), None)

# Filter job nodes
job_nodes = [n for n in nodes if n['node_type'] == 'job']


for job in tqdm(job_nodes):
    job_id = job['id']

    # Get job title via HAS_TITLE
    job_title = None
    for edge in edges:
        if edge['head_node_id'] == job_id and edge['relation_id'] == HAS_TITLE_ID:
            title_node = node_dict.get(edge['tail_node_id'])
            if title_node and title_node['node_type'] == 'job_title':
                job_title = title_node['node_name'].replace('_', ' ').title()
                break

    # Fallback: use node_name if no title found
    if not job_title:
        job_title = job['node_name'].replace('job_', '').replace('_', ' ').title()

    # Get required skills via REQUIRES_SKILL
    required_skills = []
    for edge in edges:
        if edge['head_node_id'] == job_id and edge['relation_id'] == REQUIRES_SKILL_ID:
            skill_node = node_dict.get(edge['tail_node_id'])
            if skill_node and skill_node['node_type'] == 'skill':
                skill_name = skill_node['node_name'].replace('skill_', '').replace('_', ' ').title()
                required_skills.append(skill_name)

    # Print job description
    if required_skills:
        text = f"The job has {job_title.lower()} requires the following skills: {', '.join(required_skills)}."
    else:
        text = f"The job has {job_title.lower()} has no listed required skills."
    
    embedding = model.encode(text, normalize_embeddings=True)
    blob = embedding.astype(np.float32).tobytes()
    
    # 5. Cập nhật embedding vào bảng nodes
    update_sql = "UPDATE nodes SET embedding = %s WHERE id = %s"
    cur.execute(update_sql, (blob, job_id))
    conn.commit()  # cần commit để lưu thay đổi

# Đóng kết nối
cur.close()
conn.close()




100%|██████████| 3241/3241 [02:59<00:00, 18.08it/s]
